In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from skopt import gp_minimize

In [2]:
train = pd.read_csv('../data/titanic/train.csv')
test = pd.read_csv('../data/titanic/test.csv')

In [3]:
train.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [31]:
X_train = train.drop(['PassengerId', 'Survived'], axis=1)
X_test = test.drop(['PassengerId'], axis=1)

## Features

In [32]:
def create_features(X):
    subs = {'female': 1, 'male': 0}
    X['Woman'] = X['Sex'].replace(subs)

    X['Age'] = X['Age'].fillna(X['Age'].mean())

    X['Fare'] = X['Fare'].fillna(X['Fare'].mean())

    X['Embarked'] = X['Embarked'].fillna('S')

    subs = {'S': 1, 'C': 2, 'Q':3}
    X['Harbor'] = X['Embarked'].replace(subs)

    def isVip(fare):
        if int(fare) >= 20:
            return 1
        return 0
    
    X['Vip'] = X['Fare'].apply(isVip)

    X['Kid'] = 1
    X['Kid'] = np.where(X['Age'] < 12, 1, 0)

    def getNameId(name):
        name_mapping = {'Mr': 1, 'Miss': 2, 'Mrs': 3, 'Master': 4, 'Dr': 5, 'Rev': 6, 'Major': 7, 'Col': 7, 'Mlle': 8, 'Mme': 8, 'Don': 9, 'Lady': 10, 'Countess': 10, 'Jonkheer': 10, 'Sir': 9, 'Capt': 7, 'Ms': 2}

        for n in name.split(' '):
            for key, value in name_mapping.items():
                if key in n:
                    return value
        return 0

    X['Title'] = X['Name'].apply(getNameId)

    return X
    
X_train = create_features(X_train)
X_test = create_features(X_test)

/tmp/ipykernel_88697/3897107180.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['Woman'] = X['Sex'].replace(subs)
/tmp/ipykernel_88697/3897107180.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['Harbor'] = X['Embarked'].replace(subs)
/tmp/ipykernel_88697/3897107180.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_sil

In [33]:
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Woman', 'Harbor', 'Kid', 'Title', 'Vip']

X_train = X_train[features]
X_test = X_test[features]

y_train = train['Survived']

## Padronizando os dados

In [34]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Otimizando o Random Forest

In [8]:
def train_model(parameters):
    model_rf = RandomForestClassifier(criterion=parameters[0], n_estimators=parameters[1], max_depth=parameters[2], min_samples_split=parameters[3], min_samples_leaf=parameters[4], random_state=0, n_jobs=-1)
    score = cross_val_score(model_rf, X_train, y_train, cv=10)
    mean_score = np.mean(score)
    print(f'Model mean score: {mean_score.round(4)}%')
    return -mean_score

parameters = [('entropy', 'gini'), (100, 1000), (3, 20), (2, 10), (1, 10)]
otimos = gp_minimize(train_model, parameters, random_state=0, verbose=1, n_calls=30, n_random_starts=10)

print(otimos.fun, otimos.x)

Iteration No: 1 started. Evaluating function at random point.
Model mean score: 0.8295%
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 10.0449
Function value obtained: -0.8295
Current minimum: -0.8295
Iteration No: 2 started. Evaluating function at random point.
Model mean score: 0.8294%
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 4.2549
Function value obtained: -0.8294
Current minimum: -0.8295
Iteration No: 3 started. Evaluating function at random point.
Model mean score: 0.8339%
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 8.9409
Function value obtained: -0.8339
Current minimum: -0.8339
Iteration No: 4 started. Evaluating function at random point.
Model mean score: 0.8261%
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 5.8607
Function value obtained: -0.8261
Current minimum: -0.8339
Iteration No: 5 started. Evaluating function at random point.
Model mean score: 0.8351%
Iteration No: 5 ended. Eval

## Aplicando o Random Forest

In [45]:
# Otimos: [np.str_('gini'), np.int64(975), np.int64(12), np.int64(6), np.int64(5)]

model = RandomForestClassifier(criterion='entropy', n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1, random_state=0)
# model = RandomForestClassifier(criterion='gini', n_estimators=975, max_depth=12, min_samples_split=6, min_samples_leaf=5, random_state=0)

score = cross_val_score(model, X_train, y_train, cv=10)

score = np.mean(score)

f'Model RF score: {score.round(4)}%'

'Model RF score: 0.8429%'

## Modelo final

In [190]:
model.fit(X_train, y_train)

y_pred = model.predict(X_train)

ms = confusion_matrix(y_train, y_pred)

print(f'Matriz de confusão: {ms}')

score = model.score(X_train, y_train)

print(f'Score: {score}')

Matriz de confusão: [[538  11]
 [ 50 292]]
Score: 0.9315375982042648


In [191]:
y_pred = model.predict(X_test)

submission = pd.DataFrame(test['PassengerId'])
submission['Survived'] = y_pred

submission.to_csv('submission3.csv', index=False)